# Задание для продуктовых аналитиков 

## Задание 1

#### Запросы будут написаны на proc sql (диалект SAS) и продублированы на python

In [1]:
import pandas as pd
from datetime import datetime, date, timedelta

# Загрузим данные из задания
content_watch = pd.read_excel('table1.xlsx', parse_dates=['show_date ']) 
content = pd.read_excel('table2.xlsx')

Данных немного, поэтому соединим таблицы сразу

In [ ]:
# sql
with data as (
select *
from content_watch cw 
    right join content co 
    on cw.content_id=co.content_id
)

In [2]:
# python
data = pd.merge(content_watch, content, how='right', on=['content_id '])
data

,watch_id,show_date,show_duration,platform,user_id,utm_medium,content_id,compilation_id,episode,paid_type
0,10971121570,2018-07-01 14:37:00,1340,583,1553139,organic,314472,9570,1,AVOD
1,4458319751,2018-12-01 15:00:00,12432,353,1554866,organic,314472,9570,1,AVOD
2,31382550,2018-08-02 14:39:00,1800,10,5255577,organic,314472,9570,1,AVOD
3,11254336994,2017-07-07 17:56:00,210,11,1554866,organic,314472,9570,1,AVOD
4,1231646730,2016-01-01 12:48:00,4685,11,1554866,organic,132271,NULL,NULL,SVOD
5,4212172051,2018-12-08 10:52:00,472,11,1554866,organic,314480,9570,2,AVOD
6,8909218338,2017-09-05 00:55:00,297,583,9462609,direct,127399,9570,6,TVOD
7,1904761857,2018-09-24 19:31:00,1635,9,320756,organic,127399,9570,6,TVOD
8,17947987,2018-10-30 04:45:00,854,353,1547421,referral,184673,7608,16,AVOD
9,6077839073,2017-07-12 23:58:00,4571,353,4066590,organic,222161,NULL,NULL,AVOD


### 1. На каждый день количество просмотров отдельно по монетизациям SVOD и AVOD на платформах 10 и 11 за последние 30 дней.

In [ ]:
# sql
select  date(show_date) as date, 
        paid_type, 
        platform, 
        count(watch_id)
from data
where   date(show_date)>today()-30
        and platform in (10, 11) 
        and paid_type in ("SVOD", "AVOD")
group by date(show_date), paid_type, platform

In [3]:
# python
data[ (data['platform '].isin([10,11])) &
      (data['paid_type '].isin(["SVOD ", "AVOD "])) &
      (data['show_date '] > datetime.today() - timedelta(days=360))  # фильтрация. Для наглядности взял последние 360 дней
    ].groupby([data['show_date '].apply(lambda x: x.date()), 
               'paid_type ', 
               'platform '                                           # группировка
              ])[['watch_id ']].count()                              # подсчет итогового значения

,,,watch_id
show_date,paid_type,platform,
2018-08-02,AVOD,10,1
2018-12-08,AVOD,11,1


### 2. Ежемесячный ТОП-5 сериалов и ТОП-5 единичного контента по количеству смотрящих людей.

In [ ]:
# sql
select date, content_type, id, cnt
from (select id, content_type, date, count(distinct user_id) cnt,
             rank() over (partition by date, content_type order by count(distinct user_id) desc) num
      from (select coalesce(compilation_id, content_id) as id,
                   case when compilation_id is not null then 'сериал' else 'единичный_контент' end as content_type,
                   user_id,
                   date(show_date, 'Y-M') as date
            from data)t1
      group by id, content_type, date)t2
where num<=5

In [4]:
# python
# считаем уник. пользователей для каждого контента в каждый месяц
temp = data.groupby([data['show_date '].apply(lambda x: x.strftime('%Y-%m')),
                     data.iloc[:,6:8].apply(lambda x: 'единичный_контент' if x[1] == 'NULL ' else 'сериал', axis=1),
                     data.iloc[:,6:8].apply(lambda x: x[0] if x[1] == 'NULL ' else x[1], axis=1)
                    ])[['user_id ']].nunique()
temp.index.names=['Дата','Тип контента','id контента']
# выбираем топ-5 записей для каждого типа контента
temp.sort_values(['user_id '], ascending=False).groupby(['Дата','Тип контента','id контента']).head(5)

user_id 
Дата    Тип контента      id контента          
2016-01 единичный_контент 132271              1
2017-07 единичный_контент 222161              1
        сериал            9570                1
2017-09 сериал            9570                1
2018-07 сериал            9570                1
2018-08 сериал            9570                1
2018-09 сериал            9570                1
2018-10 сериал            7608                1
2018-12 сериал            9570                1

### 3.  Список пользователей, у которых вчера был сначала просмотр с organic, а потом с referral

In [ ]:
# sql
select distinct user_id
from data t1
where date(show_date)=today()-1 and
      utm_medium='organic' and
      exists (select user_id
              from data
              where date(show_date)=today()-1 and
                    utm_medium='referral' and
                    user_id=t1.user_id and
                    show_date>t1.show_date)

In [5]:
# python
t = data[(data['show_date '].apply(lambda x: x.date()) == date.today() - timedelta(1)) &
     data['utm_medium '].isin(['referral ', 'organic '])][['show_date ', 'utm_medium ', 'user_id ']]
t = pd.merge(t[t['utm_medium ']=='referral '] , t[t['utm_medium ']=='organic '], on=['user_id '])
t[t['show_date _y'] < t['show_date _x']]['user_id '].unique()

array([], dtype=int64)

## Задание 2. Придумайте, как оценить показатели, и напишите запросы для расчёта придуманных метрик.

### 1. "Цепляемость" и "крутость" сериала. Нужна какая-то метрика, которая при наличии трёх-четырёх серий сериала позволит сравнить этот сериал по "крутости" с другими сериалами.

Крутой сериал - это тот, который смотрят быстро и до конца. Поэтому можно сделать две метрики:
1. Метрика, оценивающая объем просмотренного контента.
Расчет: среднее количество серий просмотренных одним пользователем / доступное количество серий.
Для ее расчета необходимо знать общее количетство серий в сериале.
2. Метрика, оценивающая скорость просмотра контента.
Расчет: среднее количество часов между просмотром последней и первой серии / доступное количество серий.

In [ ]:
# sql запрос для расчет второй метрики. Использую упрощение, что минимальной и максимальной датам просмотра соответствуют первая и последняя серии
select compilation_id,
        median(hours_on_episode) hours_on_episode_med
from
    (select compilation_id, 
            user_id,
            ((max(show_date) - min(show_date))/3600) / count(distinct episode) as hours_on_episode
    from data
    where compilation_id is not null
    group by compilation_id, user_id)t1
group by compilation_id

### 2. Ретеншн всех пользователей сервиса. Нужно просегментировать аудиторию, рассчитать её ретеншн по сегментам и дать рекомендации по тому, как увеличить ретеншн каждого из сегментов.

Если брать только одну метрику ретеншена, то можно посчитать rolling retention 1-го дня. Он покажет, как возвращаются люди после первого знакомства с системой. Просегментируем по дню, платформе и источнику трафика.

In [ ]:
select platform, utm_medium, date(datetime), sum(return)/count(user_id) as day_1_rolling_retention
from
    (select t1.user_id, datetime, return, platform, utm_medium
    from
        (select user_id,
                min(show_date) as datetime,
                case when max(date(show_date)) > min(date(show_date)) then 1 else 0 end as return
        from data
        group by user_id)t1
    join data on t1.user_id=data.user_id and t1.datetime=data.show_date)t2
group by platform, utm_medium, date(datetime)

Как увеличить удержание:
- в конце серии сериала показывать трейлер следующей.
- сообщать пользователю о выходе новой серии, фильма (push, рассылка)
- мотивировать к общению в комментариях (ввести карму)
- рекомендация нового контента
- спонтанные скидки 